# Tâche 6: étude de l'effet de 3 elements de symétrie différents

In [162]:
#telechargement du fichier CIF du matériau attribué
from pymatgen.ext.matproj import MPRester
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
from pymatgen.core.operations import SymmOp
import numpy as np

In [163]:
with MPRester("G0BptrHMmTXo23sw") as m:

    # Pour obtenir des informations sur un matériau (structure) avec Materials Project Id
    structure = m.get_structure_by_material_id("mp-554310")
Analyse=SpacegroupAnalyzer(structure)

Pour cette partie, en plus du site [Matérials Project](https://materialsproject.org/materials/mp-554310/#), nous utilisons [jmol Crystal Symetry Analyzer](https://www.pslc.ws/jsmol/jcse/explore.htm) afin de générer les différentes photos présentes dans ce notebook.

In [164]:
#Pour avoir la maille conventionelle, comme sur le site jmol
Anal1=SpacegroupAnalyzer(Analyse.get_conventional_standard_structure())
#Pour avoir les posotions de tous les atomes dans la maille conventionelle. Il y a 12 atomes par maille
sites=Analyse.get_conventional_standard_structure().sites
#Pour avoir toutes les opération de symétrie présentes dans la maille conventionelle. Elle sont au nombre de 36
#et ordonnées dans le même ordre que sur jmol.
Symetry=Anal1.get_symmetry_operations()

Il s'aggit ici d'analyser 3 élements de symétrie differents (autre que l'unité) chacun pour un atome différent. 

Une opération de symétrie constiste, dans l'espace cartésien, en une opération de rotation et une opération de translation représentées respectivement par une matrice 3x3 et un vecteur. Pour obtenir la nouvelle position d'un atome, on multiplie ses coordonnées par la matrice de rotation et on lui ajoute la matrice de translation.

## 1) Symétrie plan miroir

Prennons l'atome de Tl positionné en (0.67,0.33,0.83) 

In [165]:
sites[0]

PeriodicSite: Tl (2.0775, -1.1995, 18.4844) [0.6667, 0.3333, 0.8333]

Et appliquons lui une symétrie miroir (-y,-x,z) qui est la 8ème symétrie sur jmol

In [166]:
Miroir=Symetry[7]

Qui est bien la symetrie que nous voulons appliquer, en effet:

In [167]:
print("Symétrie de type " + Miroir.as_xyz_string())

Symétrie de type -y, -x, z


On observe donc que une symétrie de type plan miroir comporte une rotation mais aucune translation (vecteur nul):

In [168]:
Miroir.rotation_matrix

array([[ 0., -1.,  0.],
       [-1.,  0.,  0.],
       [ 0.,  0.,  1.]])

In [169]:
Miroir.translation_vector

array([0., 0., 0.])

Si l'on applique cette symetrie à notre atome de Tl, on obtient comme nouvelle postion:

In [170]:
Miroir.operate((0.67,0.33,0.83))

array([-0.33, -0.67,  0.83])

Pour montrer les équations cachées derrière ce résultat:

In [171]:
xyz=np.array([0.67,0.33,0.83])
sol=Miroir.rotation_matrix.dot(xyz)+Miroir.translation_vector
print(sol)

[-0.33 -0.67  0.83]


Sur cette image, le plan miroir est en vert.
On observe donc bien que l'atome garde la même coordonnée en z mais est deplacé selon x et y:

![title](https://raw.githubusercontent.com/Vahouben/lmapr1492/master/miroir.png)

## 2) Symétrie de translation

Prennons un autre atome, Bi se trouvant à l'origine (0,0,0) et appliquons lui une translation de (-1/3,1/3,1/3) 

In [172]:
sites[3]

PeriodicSite: Bi (0.0000, 0.0000, 0.0000) [0.0000, 0.0000, 0.0000]

In [174]:
Translation=SymmOp.from_xyz_string('(x-1/3,y+1/3,z+1/3)')

On observe donc de toute évidence que la matrice de rotation est la matrice unité étant donné que
par symétrie de translation, l'atome ne subit aucune rotation:

In [175]:
Translation.rotation_matrix

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])

Et nous obtenons bien le vecteur de translation attendu:

In [176]:
Translation.translation_vector

array([-0.33333333,  0.33333333,  0.33333333])

En apliquant l'opération de translation à notre atome, nous obtenons sa nouvelle position:

In [177]:
Translation.operate((0,0,0))

array([-0.33333333,  0.33333333,  0.33333333])

L'atome de Bi est représenté en mauve sur cette image, on observe que la symétrie envoie notre atome en dehors de la maille conventionelle, sa postion est donc reportée dans la maille en (0.67,033,033):

![tilte](https://raw.githubusercontent.com/Vahouben/lmapr1492/master/translation.png)

## 3) Symetrie d'inversion

Prenons maintenant l'atome de Bi positionné en (0.67,0.33,0.33) et apliquons lui une inversion.

In [178]:
sites[4]

PeriodicSite: Bi (2.0775, -1.1995, 7.3938) [0.6667, 0.3333, 0.3333]

In [179]:
Inversion=SymmOp.from_xyz_string('(-x,-y,-z)')

La matrice de rotation est bien l'opposé de la matrice unité et la matrice de translation est bein le vecteur nul.

In [180]:
Inversion.rotation_matrix

array([[-1.,  0.,  0.],
       [ 0., -1.,  0.],
       [ 0.,  0., -1.]])

In [181]:
Inversion.translation_vector

array([0., 0., 0.])

En apliquant l'opération de translation à notre atome, nous obtenons sa nouvelle position:

In [182]:
Inversion.operate((0.6667, 0.3333, 0.3333))

array([-0.6667, -0.3333, -0.3333])

L'image de notre atome se retrouve à nouveau en dehors de la maille conventionelle, sa postion est donc reportée en (0.33,0.67,0.67)

![title](https://raw.githubusercontent.com/Vahouben/lmapr1492/master/inversion.png)